## Contexto ##
#### Criar um seguro para a empresa quando o funcionário é demitido. A empresa contrata o seguro, e caso o funcionário processe a empresa, ela aciona o seguro ###
### E o seguro cobre até o limte da importânca segurada, correspondente ao que a empresa gastou com indenizações extras com o funcionário ####
### Uma mesma empresa pode fazer o seguro para vários funcionários, mas cada funcionário corresponde a uma apolice ###

## Packages and Configurations ##

In [211]:
# Basics
import pandas as pd 
import numpy as np 
import io
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

# Others
import openpyxl
import xlrd
from pathlib import Path
import os
import warnings
from math import isnan

In [212]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning) 
warnings.filterwarnings("ignore", category = FutureWarning) 
warnings.filterwarnings("ignore", category = UserWarning) 

In [213]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [214]:
path_in = '~\\Documents\\PRJ_OTIM_PRICING\\data\\raw\\'
path_out = '~\\Documents\\PRJ_OTIM_PRICING\\data\\processed\\'

## Import file ##

In [215]:
name_file_df = 'clientes_consolidado_V3.csv'
file_in = os.path.join(path_in, name_file_df)
clientes_consolidado = pd.read_csv(file_in, sep=",", encoding='UTF-8', low_memory=False)

## EDA File ##

In [216]:
clientes_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504550 entries, 0 to 504549
Data columns (total 32 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       504550 non-null  int64  
 1   CBO_Ocupação_2002        504550 non-null  int64  
 2   Faixa_Etária             504550 non-null  int64  
 3   Faixa_Hora_Contrat       504550 non-null  int64  
 4   Faixa_Remun_Dezem_(SM)   504550 non-null  int64  
 5   Escolaridade_após_2005   504550 non-null  int64  
 6   Ind_Simples              504550 non-null  int64  
 7   Município                504550 non-null  int64  
 8   Natureza_Jurídica        504550 non-null  int64  
 9   Ind_Portador_Defic       504550 non-null  int64  
 10  Raça_Cor                 504550 non-null  int64  
 11  corretora                504550 non-null  int64  
 12  Sexo_Trabalhador         504550 non-null  int64  
 13  Tamanho_Estabelecimento  504550 non-null  int64  
 14  Tipo

In [217]:
clientes_consolidado.columns

Index(['id', 'CBO_Ocupação_2002', 'Faixa_Etária', 'Faixa_Hora_Contrat',
       'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 'Ind_Simples',
       'Município', 'Natureza_Jurídica', 'Ind_Portador_Defic', 'Raça_Cor',
       'corretora', 'Sexo_Trabalhador', 'Tamanho_Estabelecimento',
       'Tipo_Admissão', 'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor',
       'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'Qtd_Hora_Contr', 'Idade',
       'Qtd_Dias_Afastamento', 'premio_input', 'Tempo_Emprego',
       'Importancia_Segurada', 'is_apolice', 'is_sinistro', 'is_cotacao',
       'Vlr_Sinistro', 'UF', 'Nome_Município'],
      dtype='object')

In [218]:
clientes_consolidado.head()

,id,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Escolaridade_após_2005,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Raça_Cor,corretora,Sexo_Trabalhador,Tamanho_Estabelecimento,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,Qtd_Hora_Contr,Idade,Qtd_Dias_Afastamento,premio_input,Tempo_Emprego,Importancia_Segurada,is_apolice,is_sinistro,is_cotacao,Vlr_Sinistro,UF,Nome_Município
0,39,10105,3,6,0,7,0,314050,2062,0,8,4744099,2,3,0,0,10,16,0,0,44.0000,18.0000,0.0000,1196.4800,10.9000,10768.3200,0.0000,0.0000,0.0000,0.0000,MG,Martinho Campos
1,40,10105,5,5,0,9,0,315180,3999,0,2,9430800,1,6,0,0,10,21,0,0,36.0000,31.0000,0.0000,2464.0986,12.4000,17248.6900,0.0000,0.0000,0.0000,0.0000,MG,Poços de Caldas
2,41,10105,5,6,0,7,0,310620,2062,0,9,5611201,2,4,0,0,10,21,0,0,44.0000,32.0000,0.0000,1196.6555,21.0000,13163.2100,0.0000,0.0000,1.0000,0.0000,MG,Belo Horizonte
3,42,10105,5,6,0,7,1,320520,2062,0,8,5611203,1,1,0,0,10,21,0,0,44.0000,36.0000,0.0000,1230.8180,7.8000,6154.0900,0.0000,0.0000,1.0000,0.0000,ES,Vila Velha
4,43,10105,5,6,0,8,0,311860,2062,0,8,5611201,2,4,0,0,10,21,0,0,44.0000,36.0000,74.0000,1338.1740,19.2000,6690.8700,0.0000,0.0000,1.0000,0.0000,MG,Contagem


In [219]:
clientes_consolidado.isnull().sum()

id                         0
CBO_Ocupação_2002          0
Faixa_Etária               0
Faixa_Hora_Contrat         0
Faixa_Remun_Dezem_(SM)     0
Escolaridade_após_2005     0
Ind_Simples                0
Município                  0
Natureza_Jurídica          0
Ind_Portador_Defic         0
Raça_Cor                   0
corretora                  0
Sexo_Trabalhador           0
Tamanho_Estabelecimento    0
Tipo_Admissão              0
Tipo_Defic                 0
Tipo_Vínculo               0
IBGE_Subsetor              0
Ind_Trab_Intermitente      0
Ind_Trab_Parcial           0
Qtd_Hora_Contr             0
Idade                      0
Qtd_Dias_Afastamento       0
premio_input               0
Tempo_Emprego              0
Importancia_Segurada       0
is_apolice                 0
is_sinistro                0
is_cotacao                 0
Vlr_Sinistro               0
UF                         0
Nome_Município             0
dtype: int64

## Make new features ##

### Add feature Comission ###

In [220]:
Natureza_Jurídica = clientes_consolidado['Natureza_Jurídica'].value_counts()
Natureza_Jurídica = pd.DataFrame(Natureza_Jurídica).reset_index()
Natureza_Jurídica['Comission'] = 0.0
Natureza_Jurídica

,Natureza_Jurídica,count,Comission
0,2062,278186,0.0000
1,2305,55175,0.0000
2,2054,45900,0.0000
3,3999,29304,0.0000
4,2135,26232,0.0000
5,2046,14944,0.0000
6,3085,10612,0.0000
7,2240,9031,0.0000
8,4081,7652,0.0000
9,4120,7628,0.0000


In [221]:
choices = [0.25, 0.15, 0.20, 0.30, 0.35, 0.10, 0.125, 0.175, 0.225, 0.10, 0.10, 0.10, 0.10, 0.10, 
           0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10]
len(choices)

28

In [222]:
for i in range(0, len(Natureza_Jurídica)):
    Natureza_Jurídica.loc[i, 'Comission'] = choices[i]

Natureza_Jurídica = Natureza_Jurídica[['Natureza_Jurídica', 'Comission']]
Natureza_Jurídica.head(), Natureza_Jurídica.tail()

(   Natureza_Jurídica  Comission
 0               2062     0.2500
 1               2305     0.1500
 2               2054     0.2000
 3               3999     0.3000
 4               2135     0.3500,
     Natureza_Jurídica  Comission
 23               2178     0.1000
 24               1023     0.1000
 25               1139     0.1000
 26               1155     0.1000
 27               1112     0.1000)

In [223]:
clientes_consolidado = clientes_consolidado.merge(Natureza_Jurídica, on=['Natureza_Jurídica'], how='left')
clientes_consolidado['Comission'].unique()

array([0.25 , 0.3  , 0.125, 0.15 , 0.1  , 0.35 , 0.2  , 0.175, 0.225])

In [224]:
clientes_consolidado['Comission'].head()

0   0.2500
1   0.3000
2   0.2500
3   0.2500
4   0.2500
Name: Comission, dtype: float64

### Empresa contrante ###

In [225]:

clientes_consolidado['CBO_Ocupação_2002'] = clientes_consolidado['CBO_Ocupação_2002'].replace('-', '', regex=True)
clientes_consolidado['CBO_Ocupação_2002'] = clientes_consolidado['CBO_Ocupação_2002'].astype(int)
clientes_consolidado['CBO'] = round(((clientes_consolidado['CBO_Ocupação_2002']/100) - ((clientes_consolidado['CBO_Ocupação_2002']%100)/100)), 2)
clientes_consolidado['CBO'] = clientes_consolidado['CBO'].astype(int)
clientes_consolidado['CBO'] = clientes_consolidado['CBO'].astype(str)
clientes_consolidado['Empresa_contrante'] = 'EC_' + clientes_consolidado['CBO']
clientes_consolidado['Empresa_contrante'].head()

0    EC_101
1    EC_101
2    EC_101
3    EC_101
4    EC_101
Name: Empresa_contrante, dtype: object

In [226]:
len(clientes_consolidado['Empresa_contrante'].value_counts())

573

### Corretora ###

In [227]:
clientes_consolidado['corretora'] = clientes_consolidado['corretora'].astype(str)
clientes_consolidado['corretora'] = 'corretora_' + clientes_consolidado['corretora']
clientes_consolidado['corretora'].head()

0    corretora_4744099
1    corretora_9430800
2    corretora_5611201
3    corretora_5611203
4    corretora_5611201
Name: corretora, dtype: object

In [228]:
len(clientes_consolidado['corretora'].value_counts())

75

### Localização geográfica - Cidades ###

In [229]:
clientes_consolidado['UF'] = clientes_consolidado['UF'].str.lower()
clientes_consolidado['Nome_Município'] = clientes_consolidado['Nome_Município'].str.lower().replace(' ', '', regex=True)
clientes_consolidado['cidades'] = clientes_consolidado['UF'] + "_" + clientes_consolidado['Nome_Município']
clientes_consolidado[['UF', 'Nome_Município', 'cidades']].head()

,UF,Nome_Município,cidades
0,mg,martinhocampos,mg_martinhocampos
1,mg,poçosdecaldas,mg_poçosdecaldas
2,mg,belohorizonte,mg_belohorizonte
3,es,vilavelha,es_vilavelha
4,mg,contagem,mg_contagem


### IBGE Sub-setor ###

In [230]:
clientes_consolidado['IBGE_Subsetor'].value_counts()

IBGE_Subsetor
16    129675
19    111236
21     73864
15     46497
20     38601
22     37778
25     20096
23     18875
13     10087
17      4766
7       4479
11      4307
18      4289
Name: count, dtype: int64

In [231]:
clientes_consolidado['IBGE_Subsetor'] = clientes_consolidado['IBGE_Subsetor'].astype(str)
clientes_consolidado['IBGE_Subsetor'] = 'IBGE_Subsetor_' + clientes_consolidado['IBGE_Subsetor']
clientes_consolidado['IBGE_Subsetor'].head()

0    IBGE_Subsetor_16
1    IBGE_Subsetor_21
2    IBGE_Subsetor_21
3    IBGE_Subsetor_21
4    IBGE_Subsetor_21
Name: IBGE_Subsetor, dtype: object

### Add News Features ###

In [232]:
clientes_consolidado['DA'] = 0.20
clientes_consolidado['Lucro'] = 0.10
IOF = 0.0738

In [233]:
clientes_consolidado['premio_comercial'] = clientes_consolidado['premio_input'] - (clientes_consolidado['premio_input'] * IOF)
clientes_consolidado['vlr_comission'] = clientes_consolidado['premio_comercial'] * clientes_consolidado['Comission']
clientes_consolidado = clientes_consolidado.eval('premio_puro = premio_comercial - (premio_comercial * (Comission + DA + Lucro))')
clientes_consolidado['Taxa_stage'] = round((clientes_consolidado['premio_comercial'] / clientes_consolidado['Importancia_Segurada']), 4)

### New premium total ###

In [234]:
b = clientes_consolidado['Taxa_stage'].unique()
b.sort()
b = b[0:11]
b

array([0.0772, 0.0842, 0.0926, 0.1029, 0.1158, 0.1323, 0.1544, 0.1852,
       0.2315, 0.2316, 0.3087])

In [235]:
c = clientes_consolidado.groupby(['corretora']).agg(N = ('id', 'count')).reset_index()
c.sort_values(by = ['N'], ascending = False, inplace = True)
c['Taxa'] = np.where(c['N'] >= 19037, 0.3087,
            np.where(c['N'] >= 12526, 0.2316,
            np.where(c['N'] >= 8184, 0.1852,
            np.where(c['N'] >= 7153, 0.1544,
            np.where(c['N'] >= 6550, 0.1323,
            np.where(c['N'] >= 4978, 0.1158,
            np.where(c['N'] >= 4289, 0.1029,
            np.where(c['N'] >= 3923, 0.0926,
            np.where(c['N'] >= 3426, 0.0842, 0.0772)))))))))
c['Taxa'].value_counts()

Taxa
0.0772    16
0.2316     8
0.1852     8
0.1158     8
0.1029     8
0.0926     8
0.0842     8
0.1544     4
0.1323     4
0.3087     3
Name: count, dtype: int64

In [236]:
c = c[['corretora', 'Taxa']]
c.head()

,corretora,Taxa
39,corretora_4930202,0.3087
41,corretora_5611201,0.3087
67,corretora_8610101,0.3087
59,corretora_8211300,0.2316
34,corretora_4781400,0.2316


In [237]:
clientes_consolidado = clientes_consolidado.rename(columns={'premio_input': 'premio_input_stage'})

In [238]:
clientes_consolidado = clientes_consolidado.merge(c, on=['corretora'], how='left')

In [239]:
clientes_consolidado = clientes_consolidado.eval('premio_input = Importancia_Segurada * Taxa')

In [240]:
print(clientes_consolidado['premio_input'].sum())
print(clientes_consolidado['premio_input_stage'].sum())
print(clientes_consolidado['premio_input'].sum() - clientes_consolidado['premio_input_stage'].sum())

1139582275.4560673
1067804610.0734398
71777665.38262749


In [241]:
# Correção desses valores devido ao novo valor do premio total
clientes_consolidado['premio_comercial'] = clientes_consolidado['premio_input'] - (clientes_consolidado['premio_input'] * IOF)
clientes_consolidado['vlr_comission'] = clientes_consolidado['premio_comercial'] * clientes_consolidado['Comission']
clientes_consolidado = clientes_consolidado.eval('premio_puro = premio_comercial - (premio_comercial * (Comission + DA + Lucro))')

In [242]:
clientes_consolidado.head()

,id,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Escolaridade_após_2005,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Raça_Cor,corretora,Sexo_Trabalhador,Tamanho_Estabelecimento,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,Qtd_Hora_Contr,Idade,Qtd_Dias_Afastamento,premio_input_stage,Tempo_Emprego,Importancia_Segurada,is_apolice,is_sinistro,is_cotacao,Vlr_Sinistro,UF,Nome_Município,Comission,CBO,Empresa_contrante,cidades,DA,Lucro,premio_comercial,vlr_comission,premio_puro,Taxa_stage,Taxa,premio_input
0,39,10105,3,6,0,7,0,314050,2062,0,8,corretora_4744099,2,3,0,0,10,IBGE_Subsetor_16,0,0,44.0000,18.0000,0.0000,1196.4800,10.9000,10768.3200,0.0000,0.0000,0.0000,0.0000,mg,martinhocampos,0.2500,101,EC_101,mg_martinhocampos,0.2000,0.1000,1539.9266,384.9817,692.9670,0.1029,0.1544,1662.6286
1,40,10105,5,5,0,9,0,315180,3999,0,2,corretora_9430800,1,6,0,0,10,IBGE_Subsetor_21,0,0,36.0000,31.0000,0.0000,2464.0986,12.4000,17248.6900,0.0000,0.0000,0.0000,0.0000,mg,poçosdecaldas,0.3000,101,EC_101,mg_poçosdecaldas,0.2000,0.1000,2958.7064,887.6119,1183.4826,0.1323,0.1852,3194.4574
2,41,10105,5,6,0,7,0,310620,2062,0,9,corretora_5611201,2,4,0,0,10,IBGE_Subsetor_21,0,0,44.0000,32.0000,0.0000,1196.6555,21.0000,13163.2100,0.0000,0.0000,1.0000,0.0000,mg,belohorizonte,0.2500,101,EC_101,mg_belohorizonte,0.2000,0.1000,3763.5979,940.8995,1693.6190,0.0842,0.3087,4063.4829
3,42,10105,5,6,0,7,1,320520,2062,0,8,corretora_5611203,1,1,0,0,10,IBGE_Subsetor_21,0,0,44.0000,36.0000,0.0000,1230.8180,7.8000,6154.0900,0.0000,0.0000,1.0000,0.0000,es,vilavelha,0.2500,101,EC_101,es_vilavelha,0.2000,0.1000,1320.1010,330.0253,594.0455,0.1852,0.2316,1425.2872
4,43,10105,5,6,0,8,0,311860,2062,0,8,corretora_5611201,2,4,0,0,10,IBGE_Subsetor_21,0,0,44.0000,36.0000,74.0000,1338.1740,19.2000,6690.8700,0.0000,0.0000,1.0000,0.0000,mg,contagem,0.2500,101,EC_101,mg_contagem,0.2000,0.1000,1913.0398,478.2599,860.8679,0.1852,0.3087,2065.4716


### Adjust of the variables ###

In [243]:
interger_cols = ['Faixa_Hora_Contrat', 'Tipo_Admissão', 'Sexo_Trabalhador', 'Faixa_Remun_Dezem_(SM)', 
                 'Escolaridade_após_2005', 'Raça_Cor', 'Tamanho_Estabelecimento',  'Tipo_Defic', 'Tipo_Vínculo']   
for  Names in interger_cols:  
    print(Names, sorted(clientes_consolidado[Names].unique()))

Faixa_Hora_Contrat [1, 2, 3, 4, 5, 6, 99]
Tipo_Admissão [0, 1, 2, 3, 4, 6]
Sexo_Trabalhador [1, 2]
Faixa_Remun_Dezem_(SM) [0]
Escolaridade_após_2005 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Raça_Cor [1, 2, 4, 6, 8, 9, 99]
Tamanho_Estabelecimento [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Tipo_Defic [0, 1, 2, 3, 4, 5, 6]
Tipo_Vínculo [10, 15, 20, 25, 30, 31, 35, 50, 55, 60, 65, 70, 75, 80, 90, 95, 96, 97]


In [244]:
clientes_consolidado['Faixa_Hora_Contrat'].value_counts()

Faixa_Hora_Contrat
6     410899
5      64793
4      12075
99      5911
1       5618
3       3772
2       1482
Name: count, dtype: int64

In [245]:
clientes_consolidado['Faixa_Hora_Contrat'] = np.where(clientes_consolidado['Faixa_Hora_Contrat'] == 99, 7, clientes_consolidado['Faixa_Hora_Contrat'])

In [246]:
clientes_consolidado['Tipo_Admissão'] = clientes_consolidado['Tipo_Admissão'] + 1

In [247]:
clientes_consolidado['Raça_Cor'] = np.where(clientes_consolidado['Raça_Cor'] == 4, 3,  
                                   np.where(clientes_consolidado['Raça_Cor'] == 6, 4,
                                   np.where(clientes_consolidado['Raça_Cor'] == 8, 5,
                                   np.where(clientes_consolidado['Raça_Cor'] == 9, 6, 
                                   np.where(clientes_consolidado['Raça_Cor'] == 99, 7, clientes_consolidado['Raça_Cor'])))))

In [248]:
clientes_consolidado['Tipo_Defic'] = clientes_consolidado['Tipo_Defic'] + 1

In [249]:
interger_cols = ['Faixa_Hora_Contrat', 'Tipo_Admissão', 'Sexo_Trabalhador', 'Faixa_Remun_Dezem_(SM)', 
                 'Escolaridade_após_2005', 'Raça_Cor', 'Tamanho_Estabelecimento',  'Tipo_Defic', 'Tipo_Vínculo']   
for  Names in interger_cols:  
    print(Names, sorted(clientes_consolidado[Names].unique()))

Faixa_Hora_Contrat [1, 2, 3, 4, 5, 6, 7]
Tipo_Admissão [1, 2, 3, 4, 5, 7]
Sexo_Trabalhador [1, 2]
Faixa_Remun_Dezem_(SM) [0]
Escolaridade_após_2005 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Raça_Cor [1, 2, 3, 4, 5, 6, 7]
Tamanho_Estabelecimento [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Tipo_Defic [1, 2, 3, 4, 5, 6, 7]
Tipo_Vínculo [10, 15, 20, 25, 30, 31, 35, 50, 55, 60, 65, 70, 75, 80, 90, 95, 96, 97]


In [250]:
booleans_cols = ['Ind_Simples', 'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'Ind_Portador_Defic']   
for  Names in booleans_cols:  
    clientes_consolidado[Names] = clientes_consolidado[Names].astype(bool) 

In [251]:
a = clientes_consolidado['Faixa_Etária'].unique()
a.sort()
a

array([2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [252]:
float_cols1 = ['Tempo_Emprego', 'premio_input', 'Importancia_Segurada', 'Vlr_Sinistro', 'Comission', 'premio_comercial', 'premio_puro', 'vlr_comission']   

for  Names1 in float_cols1:  
    clientes_consolidado[Names1] = round(clientes_consolidado[Names1], 2)

In [253]:
clientes_consolidado[['Tempo_Emprego', 'premio_input', 'Importancia_Segurada', 'Vlr_Sinistro', 'Comission', 'premio_comercial', 'premio_puro', 'vlr_comission']].head()

,Tempo_Emprego,premio_input,Importancia_Segurada,Vlr_Sinistro,Comission,premio_comercial,premio_puro,vlr_comission
0,10.9000,1662.6300,10768.3200,0.0000,0.2500,1539.9300,692.9700,384.9800
1,12.4000,3194.4600,17248.6900,0.0000,0.3000,2958.7100,1183.4800,887.6100
2,21.0000,4063.4800,13163.2100,0.0000,0.2500,3763.6000,1693.6200,940.9000
3,7.8000,1425.2900,6154.0900,0.0000,0.2500,1320.1000,594.0500,330.0300
4,19.2000,2065.4700,6690.8700,0.0000,0.2500,1913.0400,860.8700,478.2600


### Check from variables ###

In [254]:
clientes_consolidado.head()

,id,CBO_Ocupação_2002,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Escolaridade_após_2005,Ind_Simples,Município,Natureza_Jurídica,Ind_Portador_Defic,Raça_Cor,corretora,Sexo_Trabalhador,Tamanho_Estabelecimento,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,Qtd_Hora_Contr,Idade,Qtd_Dias_Afastamento,premio_input_stage,Tempo_Emprego,Importancia_Segurada,is_apolice,is_sinistro,is_cotacao,Vlr_Sinistro,UF,Nome_Município,Comission,CBO,Empresa_contrante,cidades,DA,Lucro,premio_comercial,vlr_comission,premio_puro,Taxa_stage,Taxa,premio_input
0,39,10105,3,6,0,7,False,314050,2062,False,5,corretora_4744099,2,3,1,1,10,IBGE_Subsetor_16,False,False,44.0000,18.0000,0.0000,1196.4800,10.9000,10768.3200,0.0000,0.0000,0.0000,0.0000,mg,martinhocampos,0.2500,101,EC_101,mg_martinhocampos,0.2000,0.1000,1539.9300,384.9800,692.9700,0.1029,0.1544,1662.6300
1,40,10105,5,5,0,9,False,315180,3999,False,2,corretora_9430800,1,6,1,1,10,IBGE_Subsetor_21,False,False,36.0000,31.0000,0.0000,2464.0986,12.4000,17248.6900,0.0000,0.0000,0.0000,0.0000,mg,poçosdecaldas,0.3000,101,EC_101,mg_poçosdecaldas,0.2000,0.1000,2958.7100,887.6100,1183.4800,0.1323,0.1852,3194.4600
2,41,10105,5,6,0,7,False,310620,2062,False,6,corretora_5611201,2,4,1,1,10,IBGE_Subsetor_21,False,False,44.0000,32.0000,0.0000,1196.6555,21.0000,13163.2100,0.0000,0.0000,1.0000,0.0000,mg,belohorizonte,0.2500,101,EC_101,mg_belohorizonte,0.2000,0.1000,3763.6000,940.9000,1693.6200,0.0842,0.3087,4063.4800
3,42,10105,5,6,0,7,True,320520,2062,False,5,corretora_5611203,1,1,1,1,10,IBGE_Subsetor_21,False,False,44.0000,36.0000,0.0000,1230.8180,7.8000,6154.0900,0.0000,0.0000,1.0000,0.0000,es,vilavelha,0.2500,101,EC_101,es_vilavelha,0.2000,0.1000,1320.1000,330.0300,594.0500,0.1852,0.2316,1425.2900
4,43,10105,5,6,0,8,False,311860,2062,False,5,corretora_5611201,2,4,1,1,10,IBGE_Subsetor_21,False,False,44.0000,36.0000,74.0000,1338.1740,19.2000,6690.8700,0.0000,0.0000,1.0000,0.0000,mg,contagem,0.2500,101,EC_101,mg_contagem,0.2000,0.1000,1913.0400,478.2600,860.8700,0.1852,0.3087,2065.4700


In [255]:
clientes_consolidado.columns, clientes_consolidado.shape

(Index(['id', 'CBO_Ocupação_2002', 'Faixa_Etária', 'Faixa_Hora_Contrat',
        'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 'Ind_Simples',
        'Município', 'Natureza_Jurídica', 'Ind_Portador_Defic', 'Raça_Cor',
        'corretora', 'Sexo_Trabalhador', 'Tamanho_Estabelecimento',
        'Tipo_Admissão', 'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor',
        'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'Qtd_Hora_Contr', 'Idade',
        'Qtd_Dias_Afastamento', 'premio_input_stage', 'Tempo_Emprego',
        'Importancia_Segurada', 'is_apolice', 'is_sinistro', 'is_cotacao',
        'Vlr_Sinistro', 'UF', 'Nome_Município', 'Comission', 'CBO',
        'Empresa_contrante', 'cidades', 'DA', 'Lucro', 'premio_comercial',
        'vlr_comission', 'premio_puro', 'Taxa_stage', 'Taxa', 'premio_input'],
       dtype='object'),
 (504550, 44))

In [256]:
clientes_consolidado = clientes_consolidado[['id', 'Faixa_Hora_Contrat',
        'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 'Qtd_Hora_Contr',
        'Idade', 'Ind_Simples', 
        'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor', 'corretora',
        'Sexo_Trabalhador', 'Tamanho_Estabelecimento', 'Tipo_Admissão',
        'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente',
        'Ind_Trab_Parcial', 'premio_input', 'Tempo_Emprego',
        'Importancia_Segurada', 'is_cotacao', 'is_apolice', 'is_sinistro',
        'Vlr_Sinistro', 'Comission', 
        'Empresa_contrante', 'cidades', 'premio_comercial', 'vlr_comission',
        'premio_puro', 'DA', 'Lucro', 'Faixa_Etária', 'Taxa']]

In [257]:
# Features to target enconder
print("Município:", len(clientes_consolidado['cidades'].unique()))
#print('Faixa_Etária:', len(clientes_consolidado['Desc_Faixa_Etária'].unique()))
print("IBGE_Subsetor:", len(clientes_consolidado['IBGE_Subsetor'].unique()))
print("Empresa_contrante:", len(clientes_consolidado['Empresa_contrante'].unique()))
print("corretora:", len(clientes_consolidado['corretora'].unique()))
print("Taxa:", len(clientes_consolidado['Taxa'].unique()))

Município: 994
IBGE_Subsetor: 13
Empresa_contrante: 573
corretora: 75
Taxa: 10


In [258]:
# Features to float type
float_cols = ['Qtd_Hora_Contr', 'Idade', 'Qtd_Dias_Afastamento', 'Tempo_Emprego', 'premio_input',
       'Importancia_Segurada', 'is_cotacao', 'is_apolice', 'is_sinistro', 'Vlr_Sinistro', 'Comission', 'premio_comercial', 'premio_puro', 'vlr_comission', 'Taxa']   
clientes_consolidado[float_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Qtd_Hora_Contr,504550.0000,41.5260,7.2788,0.0000,44.0000,44.0000,44.0000,44.0000
Idade,504550.0000,36.3316,11.2098,15.0000,27.0000,35.0000,43.0000,99.0000
Qtd_Dias_Afastamento,504550.0000,4.5790,20.7523,0.0000,0.0000,0.0000,0.0000,365.0000
Tempo_Emprego,504550.0000,30.7950,45.4935,0.0000,6.1000,14.9000,35.3000,595.9000
premio_input,504550.0000,2258.6112,3058.2911,278.6600,949.4525,1557.9500,2679.8175,481719.6700
Importancia_Segurada,504550.0000,13115.3753,16288.7621,3609.2900,6354.4925,9866.2350,14815.4750,1900394.7200
is_cotacao,504550.0000,0.5000,0.5000,0.0000,0.0000,1.0000,1.0000,1.0000
is_apolice,504550.0000,0.1076,0.3099,0.0000,0.0000,0.0000,0.0000,1.0000
is_sinistro,504550.0000,0.0150,0.1216,0.0000,0.0000,0.0000,0.0000,1.0000
Vlr_Sinistro,504550.0000,182.7162,2065.6570,0.0000,0.0000,0.0000,0.0000,219843.9500


In [259]:
interger_cols = ['Faixa_Hora_Contrat', 'Tipo_Admissão', 'Sexo_Trabalhador', 'Faixa_Etária',
       'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 'Raça_Cor', 'Tamanho_Estabelecimento', 'Tipo_Defic', 'Tipo_Vínculo', 'Taxa']   
for  Names in interger_cols:  
    print(clientes_consolidado[Names].value_counts())

Faixa_Hora_Contrat
6    410899
5     64793
4     12075
7      5911
1      5618
3      3772
2      1482
Name: count, dtype: int64
Tipo_Admissão
1    353120
3    131081
4     10080
2      5356
5      4748
7       165
Name: count, dtype: int64
Sexo_Trabalhador
1    310792
2    193758
Name: count, dtype: int64
Faixa_Etária
5    157613
6    107602
4     95567
3     72206
7     64134
8      7157
2       271
Name: count, dtype: int64
Faixa_Remun_Dezem_(SM)
0    504550
Name: count, dtype: int64
Escolaridade_após_2005
7     263858
9      78411
5      44430
6      37440
4      26241
8      24904
2      12222
3      11667
10      2514
1       1709
11      1154
Name: count, dtype: int64
Raça_Cor
2    186384
5    184338
6     80335
3     47688
4      3702
1      1189
7       914
Name: count, dtype: int64
Tamanho_Estabelecimento
5     78358
4     70485
2     60321
3     57445
7     49906
6     48519
10    38183
1     36472
8     34205
9     30656
Name: count, dtype: int64
Tipo_Defic
1    500602
2   

In [260]:
booleans_cols = ['Ind_Simples', 'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'Ind_Portador_Defic']   
for  Names in booleans_cols:  
    print(clientes_consolidado[Names].value_counts())

Ind_Simples
False    348224
True     156326
Name: count, dtype: int64
Ind_Trab_Intermitente
False    502805
True       1745
Name: count, dtype: int64
Ind_Trab_Parcial
False    501599
True       2951
Name: count, dtype: int64
Ind_Portador_Defic
False    500602
True       3948
Name: count, dtype: int64


In [261]:
clientes_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504550 entries, 0 to 504549
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       504550 non-null  int64  
 1   Faixa_Hora_Contrat       504550 non-null  int64  
 2   Faixa_Remun_Dezem_(SM)   504550 non-null  int64  
 3   Escolaridade_após_2005   504550 non-null  int64  
 4   Qtd_Hora_Contr           504550 non-null  float64
 5   Idade                    504550 non-null  float64
 6   Ind_Simples              504550 non-null  bool   
 7   Ind_Portador_Defic       504550 non-null  bool   
 8   Qtd_Dias_Afastamento     504550 non-null  float64
 9   Raça_Cor                 504550 non-null  int64  
 10  corretora                504550 non-null  object 
 11  Sexo_Trabalhador         504550 non-null  int64  
 12  Tamanho_Estabelecimento  504550 non-null  int64  
 13  Tipo_Admissão            504550 non-null  int64  
 14  Tipo

### Export dataframe ###

In [262]:
name_file_df = 'dataframe_processed_03.csv'
file_out = os.path.join(path_out, name_file_df)
clientes_consolidado.to_csv(file_out, index=False)